# Пропуски в данных
Часто данные в реальных приложениям имеют пропуски, т.е. некоторые значения могут быть пустыми. При анализе данных почти всегда приходиться иметь дело с такими наборами данных и необходимо корректно их обрабатывать.

В NumPy для пустых значений чисел используется специальное значение `np.nan`. pandas, будучи создан поверх NumPy, тоже использует `np.nan` в качестве значения для пропущенных значений чисел. Для всех остальных типов данных используется значение `None` в Python.

In [2]:
import numpy as np
import pandas as pd

## Пропуски в NumPy
В начале необходимо посмотреть как NumPy работает с пропусками. Во-первых, если в массиве NumPy встречается значение `None`, то тип этого массива устанавливается как `object`

In [5]:
objects = np.array([1, None, 3, 4])
objects.dtype

dtype('O')

Во-вторых, над массивом с типом `object` нельзя выполнять агрегирующие операции (`min`, `max`, `sum` и т.д.)

Это приводит к тому, что все опреации выполняются на уровне Python. В таких случаях вычисления могут выполняться на порядок медленно

In [6]:
%timeit np.arange(1E6, dtype='object').sum()
%timeit np.arange(1E6, dtype='int').sum()

10 loops, best of 3: 55.5 ms per loop
1000 loops, best of 3: 1.74 ms per loop


Во-вторых, над массивом с типом `object` нельзя выполнять агрегирующие операции (`min`, `max`, `sum` и т.д.)

In [8]:
objects.max()

TypeError: '>=' not supported between instances of 'int' and 'NoneType'

По этой причине для числовых массивов необходимо использовать `np.nan` для указания пропусков

In [9]:
numbers = np.array([1, np.nan, 3, 4])
numbers.dtype

dtype('float64')

Любая арифметическая операция с `np.nan` выдает результат `np.nan`. Поэтому любые агрегирующие функции NumPy также выдают `np.nan` если есть хотя бы один пропуск в массиве

In [11]:
numbers.min(), numbers.max(), numbers.sum()

(nan, nan, nan)

Чтобы игнорировать значение `np.nan` при агрегации есть специальные версии функций `nanmax`, `nanmin`, `nansum` и т.д.

In [13]:
np.nanmin(numbers), np.nanmax(numbers), np.nansum(numbers)

(1.0, 4.0, 8.0)

## Пропуски в pandas
Pandas, в отличии от NumPy, всегда старается использовать наиболее подходящее значение для пропуска. Если в списке чисел указать значение `None`, то оно будет интерпретироваться как `np.nan`

In [15]:
pd.Series([1, None])

0    1.0
1    NaN
dtype: float64

Значение NaN определен стандартом IEEE для чисел с плавающей запятой, поэтому `np.nan` может быть использован только с числами с плавающей запятой. Если использовать `np.nan` для целых чисел, то тип массива автоматически поменяется на `float`

In [19]:
numbers = pd.Series(range(4), dtype='int')
numbers

0    0
1    1
2    2
3    3
dtype: int32

In [20]:
numbers[3] = None
numbers

0    0.0
1    1.0
2    2.0
3    NaN
dtype: float64

### Определение и удаление пропусков 
Для определения пустых значений можно использовать функцию `isnull`

In [23]:
numbers = pd.Series([0, None, 2, np.nan, 4])
numbers.isnull()

0    False
1     True
2    False
3     True
4    False
dtype: bool

Для определения не пустых значений можно использовать `notnull`

In [24]:
numbers.notnull()

0     True
1    False
2     True
3    False
4     True
dtype: bool

Можно использовать эти булевые массиве в качестве маски для выбора не пустых значений

In [25]:
numbers[numbers.notnull()]

0    0.0
2    2.0
4    4.0
dtype: float64

Помимо вышеуказанного способа для удаления пустых значений есть метод `dropna`. Посмотрим его работу с `DataFrame`

In [47]:
df = pd.DataFrame(np.arange(16).reshape((4, 4)))
df.iloc[0, 0] = np.nan
df.iloc[2, 2] = np.nan
df.iloc[1, 2] = np.nan
df

,0,1,2,3
0,NaN,1,2.0,3
1,4.0,5,NaN,7
2,8.0,9,NaN,11
3,12.0,13,14.0,15


In [40]:
df.dropna()

,0,1,2,3
3,12.0,13,14.0,15


Можно указать ось (`axis`, строки или колонки) для удаления пустых значений и пороговое количество не пустых значений (`thresh`) для сохранения строки/колонки

In [50]:
df.dropna(axis=1, thresh=3)

,0,1,3
0,NaN,1,3
1,4.0,5,7
2,8.0,9,11
3,12.0,13,15


### Заполнение пропусков значениями
Вместо удаления данных с пустыми значениями иногда имеет смысл заполнить их какими нибудь значениями. Для этого в pandas есть метод `fillna`. Например, можно заполнить все пустые значения нулями

In [51]:
df.fillna(0)

,0,1,2,3
0,0.0,1,2.0,3
1,4.0,5,0.0,7
2,8.0,9,0.0,11
3,12.0,13,14.0,15


В `fillna` есть параметр `method`, который принимает значения `ffill` и `bfill`. 

Метод `ffill` (forward fill) заполняет пустые значения предыдущим непустым значением

In [52]:
df.fillna(method='ffill')

,0,1,2,3
0,NaN,1,2.0,3
1,4.0,5,2.0,7
2,8.0,9,2.0,11
3,12.0,13,14.0,15


Метод `bfill` заполняет пустые значения следующим непустым значением

In [53]:
df.fillna(method='bfill')

,0,1,2,3
0,4.0,1,2.0,3
1,4.0,5,14.0,7
2,8.0,9,14.0,11
3,12.0,13,14.0,15


Ингода имеет смысл заменить все пустые значения средним значением по строке или колонке

In [57]:
df.mean()

0    8.0
1    7.0
2    8.0
3    9.0
dtype: float64

In [58]:
df.fillna(df.mean())

,0,1,2,3
0,8.0,1,2.0,3
1,4.0,5,8.0,7
2,8.0,9,8.0,11
3,12.0,13,14.0,15
